# Potential Organ Donor Cohort Identifier

In [ ]:
import sys 
import os
import polars as pl 
import matplotlib.pyplot as plt
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0, project_root)
from utils.config import config
from utils.io import read_data
from utils.strobe_diagram import create_consort_diagram
from clifpy.utils.stitching_encounters import stitch_encounters

In [ ]:
site_name = config['site_name']
tables_path = config['tables_path']
file_type = config['file_type']
print(f"Site Name: {site_name}")
print(f"Tables Path: {tables_path}")
print(f"File Type: {file_type}")

In [ ]:
# read required tables
adt_filepath = f"{tables_path}/clif_adt.{file_type}"
hospitalization_filepath = f"{tables_path}/clif_hospitalization.{file_type}"
patient_filepath = f"{tables_path}/clif_patient.{file_type}"
resp_filepath = f"{tables_path}/clif_respiratory_support.{file_type}"
labs_filepath = f"{tables_path}/clif_labs.{file_type}"
micro_culture_filepath = f"{tables_path}/clif_microbiology_culture.{file_type}"
crrt_filepath = f"{tables_path}/clif_crrt_therapy.{file_type}"


adt_df = read_data(adt_filepath, file_type)
hospitalization_df = read_data(hospitalization_filepath, file_type)
patient_df = read_data(patient_filepath, file_type)
resp_df = read_data(resp_filepath, file_type)
labs_df = read_data(labs_filepath, file_type)
micro_culture = read_data(micro_culture_filepath, file_type)
crrt_therapy = read_data(crrt_filepath, file_type)

total_patients = hospitalization_df["patient_id"].n_unique()

In [ ]:
hosp_stitched, adt_stitched, encounter_mapping = stitch_encounters(
      hospitalization=hospitalization_df.to_pandas(),
      adt=adt_df.to_pandas(),
      time_interval=12
  )

hosp_stitched = pl.from_pandas(hosp_stitched)
adt_stitched = pl.from_pandas(adt_stitched)
encounter_mapping = pl.from_pandas(encounter_mapping)

In [ ]:
# Step 1: Get the admission dates for the expired
# filter to patients that expired or discharged to hospice 
expired_encounters_df = hosp_stitched.filter(
    pl.col('discharge_category').str.to_lowercase() == 'expired'
)
expired_hospitalizations = (
    expired_encounters_df
    .select([
        'patient_id',
        'hospitalization_id',
        'encounter_block',
        'admission_dttm',
        'discharge_dttm'  # discharge datetime for the death hospitalization
    ])
    .unique()
)
expired_patients_n = expired_hospitalizations["patient_id"].n_unique()

In [ ]:
# Step 2- On invasive mechanical ventilation at or within 48h of death.

# Expired patients ever on IMV
imv_resp_encounters = resp_df.filter(pl.col("device_category")=="IMV") 
imv_expired = expired_hospitalizations.join(imv_resp_encounters.select(["hospitalization_id", "recorded_dttm"]), on = "hospitalization_id", how = "inner")

resp_expired_latest_recorded_imv = (
    imv_expired
    .sort("recorded_dttm", descending=True)
    .group_by("patient_id")
    .agg(pl.all().first())
)

resp_expired_imv_hrs = resp_expired_latest_recorded_imv.with_columns(
    (
        (pl.col("discharge_dttm") - pl.col("recorded_dttm")).dt.total_seconds() / 3600
    ).alias("hr_2death_last_imv")
)

# Filter to patients who were on IMV at death or before 48hrs of death 
resp_expired_cohort = resp_expired_imv_hrs.filter(pl.col('hr_2death_last_imv')<=48)

imv_expired_patients = imv_expired["patient_id"].n_unique()
imv_after_expire = resp_expired_imv_hrs.filter(pl.col('hr_2death_last_imv') <= 0)["patient_id"].n_unique()
imv_48hr_expire = resp_expired_cohort["patient_id"].n_unique()


In [ ]:
# Step 3 - Pass the potential organ quality assessment check (independent assessment) using labs from the 48 hours prior to death

# Step 3A - Kidney: creatinine <4  AND not on CRRT
# Filter out encounters who ever received crrt treatment
organ_assess_cohort = resp_expired_cohort.join(crrt_therapy.select(["hospitalization_id"]), on="hospitalization_id", how="anti")
patients_crrt_n = len(set(resp_expired_cohort["patient_id"]) - set(organ_assess_cohort["patient_id"]))

# Filter out creatinine >4
creatinine = labs_df.filter(pl.col("lab_category").is_in(["creatinine"]))

# Perform an inner join to actually bring in the matching creatinine rows/columns
relevant_creatinine = organ_assess_cohort.join(
    creatinine.select(["hospitalization_id", "lab_collect_dttm", "lab_value_numeric"]),
    on="hospitalization_id",
    how="left"
)

# Step 2: Get the latest creatinine per hospitalization
latest_creatinine = (
    relevant_creatinine
    .sort("lab_collect_dttm")  # Sort by collection time
    .group_by("hospitalization_id")
    .agg([
        pl.col("lab_collect_dttm").last().alias("latest_creatinine_dttm"),
        pl.col("lab_value_numeric").last().alias("latest_creatinine_value")
    ])
)

# Join back to organ_assess_cohort if needed
kidney_assess = (
    organ_assess_cohort
    .join(
        latest_creatinine,
        on="hospitalization_id",
        how="left"
    )
    .with_columns(
        (
            (pl.col("discharge_dttm") - pl.col("latest_creatinine_dttm")).dt.total_seconds() / 3600
        ).alias("hrs_before_discharge_latest_creatinine")
    )
)
# Filter out rows where latest_creatinine_value > 4 AND hrs_before_discharge_latest_creatinine <= 48
kidney_eligible = kidney_assess.filter(
    ~((pl.col("latest_creatinine_value") > 4) & (pl.col("hrs_before_discharge_latest_creatinine") <= 48))
)

# Capture the number of unique patients that meet the criteria and are filtered out
filtered_out_patients_count = kidney_assess.filter(
    (pl.col("latest_creatinine_value") > 4) & (pl.col("hrs_before_discharge_latest_creatinine") <= 48)
)["patient_id"].n_unique()
eligible_kidney_n = kidney_eligible["patient_id"].n_unique()


# Step 3B - Liver: Total bilirubin < 4, AST < 700, AND ALT < 700

# Filter liver enzymes and bilirubin
liver_labs = labs_df.filter(
    pl.col("lab_category").is_in(["bilirubin_total", "ast", "alt"])
)

# Bring in matching liver lab rows/columns
relevant_liver = organ_assess_cohort.join(
    liver_labs.select(["hospitalization_id", "lab_category", "lab_collect_dttm", "lab_value_numeric"]),
    on="hospitalization_id",
    how="left"
)

# Get the latest value for each of the three labs per hospitalization
latest_liver_labs = (
    relevant_liver
    .sort("lab_collect_dttm")
    .group_by(["hospitalization_id", "lab_category"])
    .agg([
        pl.col("lab_collect_dttm").last().alias("latest_lab_dttm"),
        pl.col("lab_value_numeric").last().alias("latest_lab_value")
    ])
    .pivot(
        values="latest_lab_value",
        index="hospitalization_id",
        on="lab_category"
    )
    # Renaming columns for clarity
    .rename({
        "bilirubin_total": "latest_total_bilirubin",
        "ast": "latest_ast",
        "alt": "latest_alt"
    })
)
latest_liver_lab_times = (
    relevant_liver
    .sort("lab_collect_dttm")
    .group_by(["hospitalization_id", "lab_category"])
    .agg([
        pl.col("lab_collect_dttm").last().alias("latest_lab_dttm"),
    ])
    .pivot(
        values="latest_lab_dttm",
        index="hospitalization_id",
        on="lab_category"
    )
    .rename({
        "bilirubin_total": "latest_bili_collect_dttm",
        "ast": "latest_ast_collect_dttm",
        "alt": "latest_alt_collect_dttm"
    })
)

# Now join both the value and timestamp pivots to organ_assess_cohort
liver_assess = (
    organ_assess_cohort
    .join(
        latest_liver_labs,  # contains the last values by category
        on="hospitalization_id",
        how="left"
    )
    .join(
        latest_liver_lab_times,  # contains the last lab collection times by category
        on="hospitalization_id",
        how="left"
    )
    .with_columns([
        ((pl.col("discharge_dttm") - pl.col("latest_bili_collect_dttm")).dt.total_seconds() / 3600).alias("hrs_before_discharge_latest_bili"),
        ((pl.col("discharge_dttm") - pl.col("latest_ast_collect_dttm")).dt.total_seconds() / 3600).alias("hrs_before_discharge_latest_ast"),
        ((pl.col("discharge_dttm") - pl.col("latest_alt_collect_dttm")).dt.total_seconds() / 3600).alias("hrs_before_discharge_latest_alt"),
    ])
)

liver_eligible = liver_assess.filter(
    (
        ((pl.col("latest_total_bilirubin") < 4) & (pl.col("hrs_before_discharge_latest_bili") <= 48))
        |
        ((pl.col("latest_ast") < 700) & (pl.col("hrs_before_discharge_latest_ast") <= 48))
        |
        ((pl.col("latest_alt") < 700) & (pl.col("hrs_before_discharge_latest_alt") <= 48))
    )
)

# Capture number of unique patients filtered out by each liver exclusion criteria
n_bili_gt4 = liver_assess.filter(
    (pl.col("latest_total_bilirubin") >= 4) & (pl.col("hrs_before_discharge_latest_bili") <= 48)
)["patient_id"].n_unique()

n_ast_gt700 = liver_assess.filter(
    (pl.col("latest_ast") >= 700) & (pl.col("hrs_before_discharge_latest_ast") <= 48)
)["patient_id"].n_unique()

n_alt_gt700 = liver_assess.filter(
    (pl.col("latest_alt") >= 700) & (pl.col("hrs_before_discharge_latest_alt") <= 48)
)["patient_id"].n_unique()

eligible_liver_n = liver_eligible["patient_id"].n_unique()

# Fix ShapeError: Align kidney_eligible and liver_eligible to same columns before concat
common_columns = ["hospitalization_id", "patient_id", "encounter_block", "discharge_dttm"]
overall_organ_eligible = (
    pl.concat([
        kidney_eligible.select(common_columns),
        liver_eligible.select(common_columns)
    ])
    .unique()
)
overall_organ_eligible_patients = overall_organ_eligible["patient_id"].n_unique()


In [ ]:
# Creatinine missing count (patients in organ_assess_cohort with NO creatinine in last 48h)
creatinine_missing_patients_n = kidney_assess.filter(
    (pl.col("latest_creatinine_value").is_null()) |
    (pl.col("hrs_before_discharge_latest_creatinine") > 48)
)["patient_id"].n_unique()
print(f"Number of patients missing creatinine labs in last 48h: {creatinine_missing_patients_n}")

# Missing lab counts for liver labs in last 48 hours

bilirubin_missing_patients_n = liver_assess.filter(
    (pl.col("latest_total_bilirubin").is_null()) |
    (pl.col("hrs_before_discharge_latest_bili") > 48)
)["patient_id"].n_unique()
print(f"Number of patients missing bilirubin labs in last 48h: {bilirubin_missing_patients_n}")

ast_missing_patients_n = liver_assess.filter(
    (pl.col("latest_ast").is_null()) |
    (pl.col("hrs_before_discharge_latest_ast") > 48)
)["patient_id"].n_unique()
print(f"Number of patients missing AST labs in last 48h: {ast_missing_patients_n}")

alt_missing_patients_n = liver_assess.filter(
    (pl.col("latest_alt").is_null()) |
    (pl.col("hrs_before_discharge_latest_alt") > 48)
)["patient_id"].n_unique()
print(f"Number of patients missing ALT labs in last 48h: {alt_missing_patients_n}")

# Number of patients missing ALL liver lab values (bilirubin, ast, alt) in the last 48 hours
missing_any_liver_lab_n = liver_assess.filter(
    ((pl.col("latest_total_bilirubin").is_null()) | (pl.col("hrs_before_discharge_latest_bili") > 48)) &
    ((pl.col("latest_ast").is_null()) | (pl.col("hrs_before_discharge_latest_ast") > 48)) &
    ((pl.col("latest_alt").is_null()) | (pl.col("hrs_before_discharge_latest_alt") > 48))
)["patient_id"].n_unique()
print(f"Number of patients missing ALL bilirubin, AST, and ALT labs in last 48h: {missing_any_liver_lab_n}")


In [ ]:
# Step - 4 CONTRAINDICATIONS

# Step 4A - No positive blood cultures within 48hrs

# Step 4B - History of Cancer or Sepsis

# Step 4C - Age < 75
relevant_cohort_with_birth = overall_organ_eligible.join(
    patient_df.select(['patient_id', 'birth_date']),
    on='patient_id',
    how='left'
    )

# Calculate age at death as (discharge_dttm - birth_date) in years (using .dt.total_days()/365.25)
relevant_cohort_with_deathage = relevant_cohort_with_birth.with_columns(
(
    (pl.col('discharge_dttm') - pl.col('birth_date')).dt.total_days() / 365.25
).alias('age_at_death')
)

age_relevant_cohort = relevant_cohort_with_deathage.filter(
    (pl.col('age_at_death') >= 18) & (pl.col('age_at_death') <= 75)
)
age_relevant_cohort_n = age_relevant_cohort["patient_id"].n_unique()

# Final cohort

In [ ]:
# relevant_cohort_with_deathage.select([
#     "patient_id", 
#     "hospitalization_id", 
#     "encounter_block", 
#     "age_at_death"
# ]).unique().write_parquet("../output/intermediate/relevant_cohort_with_deathage.parquet")

In [ ]:

steps = [
    {
        'label': '\nAll Patients\n',
        'n': total_patients,
        'color': 'blue'
    },
    {
        'label': 'Deceased Patients',
        'note': '(not including Hospice)',
        'n': expired_patients_n,
        'color': 'blue',
    },
    {
        'label': '\nEver on IMV',
        'n': imv_expired_patients,
        'color': 'blue'
    },
    {
        'label': '\nOn IMV at death \n or 48hrs prior\n',
        'n': imv_48hr_expire,
        'color': 'blue',
        'split': [
            {
                'label': 'Patients on IMV\n after recorded death time\n',
                'n': imv_after_expire,
                'color': 'red'
            },
            {
                'label': 'Patients on IMV within\n 48hrs before or at death\n',
                'note': "Deceased Patients who were IMV on since 48hrs before death",
                'n': imv_48hr_expire - imv_after_expire,
                'color': 'red'
            }
        ]
    },
    {
        'label': '\nPatients passing one or \nboth organ quality check\n',
        'n': overall_organ_eligible_patients,
        'color': 'blue',
                'split': [
            {
                'label': 'Eligible Kidneys',
                'note': f" excluded on crrt {patients_crrt_n} followed by\n creatinine >4 {filtered_out_patients_count}",
                'n': eligible_kidney_n,
                'color': 'red'
            },
            {
                'label': 'Eligible Livers',
                'note': f"bilirubin > 4: {n_bili_gt4}\nAST> 700: {n_ast_gt700}\nALT>700: {n_alt_gt700}\n ",
                'n': eligible_liver_n,
                'color': 'red'
            }
        ]

    },
    {
        'label': '\nPatients aged 75 or less\n at death\n',
        'n': age_relevant_cohort_n,
        'color': 'blue'
    },
]


fig = create_consort_diagram(
    steps,
    title="COHORT SELECTION Flow Diagram",
    subtitle="Potential Organ Donors"
)
fig.savefig("../output/final/cohort_strobe.png", bbox_inches="tight", dpi=300)
plt.show()